## Pembenaman

Dalam contoh sebelumnya, kita telah bekerja dengan vektor bag-of-words berdimensi tinggi dengan panjang `vocab_size`, dan kita secara eksplisit menukar daripada vektor perwakilan kedudukan berdimensi rendah kepada perwakilan one-hot yang jarang. Perwakilan one-hot ini tidak cekap dari segi memori, selain itu, setiap perkataan dianggap secara bebas antara satu sama lain, iaitu vektor one-hot yang dikodkan tidak menyatakan sebarang persamaan semantik antara perkataan.

Dalam unit ini, kita akan terus meneroka set data **News AG**. Untuk memulakan, mari kita muatkan data dan dapatkan beberapa definisi daripada buku nota sebelumnya.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Apakah itu embedding?

Idea **embedding** adalah untuk mewakili perkataan dengan vektor padat berdimensi rendah, yang mencerminkan makna semantik sesuatu perkataan. Kita akan bincangkan kemudian bagaimana untuk membina embedding perkataan yang bermakna, tetapi buat masa ini, anggaplah embedding sebagai cara untuk mengurangkan dimensi vektor perkataan.

Jadi, lapisan embedding akan mengambil satu perkataan sebagai input, dan menghasilkan vektor output dengan `embedding_size` yang ditentukan. Dalam erti kata lain, ia sangat mirip dengan lapisan `Linear`, tetapi bukannya mengambil vektor yang dikodkan satu-panas (one-hot encoded), ia akan dapat mengambil nombor perkataan sebagai input.

Dengan menggunakan lapisan embedding sebagai lapisan pertama dalam rangkaian kita, kita boleh beralih daripada model bag-of-words kepada model **embedding bag**, di mana kita mula-mula menukar setiap perkataan dalam teks kita kepada embedding yang sepadan, dan kemudian mengira beberapa fungsi agregat ke atas semua embedding tersebut, seperti `sum`, `average` atau `max`.

![Imej menunjukkan pengklasifikasi embedding untuk lima perkataan dalam urutan.](../../../../../translated_images/ms/embedding-classifier-example.b77f021a7ee67eee.webp)

Rangkaian neural pengklasifikasi kita akan bermula dengan lapisan embedding, kemudian lapisan agregasi, dan pengklasifikasi linear di atasnya:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Mengendalikan Saiz Urutan Pembolehubah

Akibat daripada seni bina ini, minibatch untuk rangkaian kita perlu dicipta dengan cara tertentu. Dalam unit sebelumnya, semasa menggunakan bag-of-words, semua tensor BoW dalam satu minibatch mempunyai saiz yang sama `vocab_size`, tanpa mengira panjang sebenar urutan teks kita. Apabila kita beralih kepada embedding perkataan, kita akan berakhir dengan bilangan perkataan yang berbeza dalam setiap sampel teks, dan apabila menggabungkan sampel-sampel tersebut ke dalam minibatch, kita perlu menggunakan padding.

Ini boleh dilakukan dengan menggunakan teknik yang sama iaitu menyediakan fungsi `collate_fn` kepada datasource:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Melatih pengklasifikasi embedding

Sekarang setelah kita menetapkan dataloader yang sesuai, kita boleh melatih model menggunakan fungsi latihan yang telah kita tetapkan dalam unit sebelumnya:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Nota**: Kami hanya melatih untuk 25k rekod di sini (kurang daripada satu epoch penuh) demi menjimatkan masa, tetapi anda boleh meneruskan latihan, menulis fungsi untuk melatih beberapa epoch, dan bereksperimen dengan parameter kadar pembelajaran untuk mencapai ketepatan yang lebih tinggi. Anda sepatutnya dapat mencapai ketepatan sekitar 90%.


### Lapisan EmbeddingBag dan Representasi Jujukan Panjang Berubah

Dalam seni bina sebelumnya, kami perlu menambah semua jujukan kepada panjang yang sama untuk dimuatkan ke dalam minibatch. Ini bukan cara yang paling efisien untuk mewakili jujukan panjang berubah - pendekatan lain adalah menggunakan vektor **offset**, yang akan menyimpan offset semua jujukan yang disimpan dalam satu vektor besar.

![Imej menunjukkan representasi jujukan offset](../../../../../translated_images/ms/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Nota**: Dalam gambar di atas, kami menunjukkan jujukan watak, tetapi dalam contoh kami, kami bekerja dengan jujukan perkataan. Walau bagaimanapun, prinsip umum mewakili jujukan dengan vektor offset tetap sama.

Untuk bekerja dengan representasi offset, kami menggunakan lapisan [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Ia serupa dengan `Embedding`, tetapi ia mengambil vektor kandungan dan vektor offset sebagai input, dan ia juga termasuk lapisan purata, yang boleh menjadi `mean`, `sum` atau `max`.

Berikut adalah rangkaian yang diubah suai yang menggunakan `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Untuk menyediakan set data untuk latihan, kita perlu menyediakan fungsi penukaran yang akan menyediakan vektor anjakan:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Perhatikan bahawa tidak seperti dalam semua contoh sebelumnya, rangkaian kita kini menerima dua parameter: vektor data dan vektor offset, yang mempunyai saiz berbeza. Begitu juga, pemuat data kita juga memberikan kita 3 nilai bukannya 2: kedua-dua vektor teks dan offset disediakan sebagai ciri. Oleh itu, kita perlu sedikit menyesuaikan fungsi latihan kita untuk mengurus perkara tersebut:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Perwakilan Semantik: Word2Vec

Dalam contoh sebelumnya, lapisan embedding model belajar memetakan perkataan kepada perwakilan vektor, namun, perwakilan ini tidak mempunyai banyak makna semantik. Adalah lebih baik jika kita dapat belajar perwakilan vektor sedemikian rupa, di mana perkataan yang serupa atau sinonim akan sepadan dengan vektor yang dekat antara satu sama lain berdasarkan jarak vektor tertentu (contohnya, jarak Euclidean).

Untuk mencapai ini, kita perlu melatih model embedding kita terlebih dahulu pada koleksi teks yang besar dengan cara tertentu. Salah satu kaedah pertama untuk melatih perwakilan semantik ini dipanggil [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ia berdasarkan dua seni bina utama yang digunakan untuk menghasilkan perwakilan teragih bagi perkataan:

 - **Continuous bag-of-words** (CBoW) — dalam seni bina ini, kita melatih model untuk meramalkan satu perkataan berdasarkan konteks sekeliling. Diberikan ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, matlamat model adalah untuk meramalkan $W_0$ daripada $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** adalah bertentangan dengan CBoW. Model ini menggunakan tetingkap perkataan konteks sekeliling untuk meramalkan perkataan semasa.

CBoW lebih pantas, manakala skip-gram lebih perlahan, tetapi lebih baik dalam mewakili perkataan yang jarang digunakan.

![Imej menunjukkan kedua-dua algoritma CBoW dan Skip-Gram untuk menukar perkataan kepada vektor.](../../../../../translated_images/ms/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Untuk mencuba embedding word2vec yang telah dilatih terlebih dahulu pada dataset Google News, kita boleh menggunakan perpustakaan **gensim**. Di bawah ini, kita mencari perkataan yang paling serupa dengan 'neural'

> **Nota:** Apabila anda mula-mula mencipta vektor perkataan, proses memuat turun mungkin mengambil sedikit masa!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Kita juga boleh mengira penjelmaan vektor daripada perkataan, untuk digunakan dalam melatih model klasifikasi (kami hanya menunjukkan 20 komponen pertama vektor untuk kejelasan):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Perkara hebat tentang pengekodan semantik ialah anda boleh memanipulasi pengekodan vektor untuk mengubah semantik. Sebagai contoh, kita boleh meminta untuk mencari perkataan, yang pengekodan vektornya sedekat mungkin dengan perkataan *raja* dan *wanita*, dan sejauh mungkin dari perkataan *lelaki*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Kedua-dua CBoW dan Skip-Grams adalah "embedding ramalan", kerana mereka hanya mengambil kira konteks tempatan. Word2Vec tidak memanfaatkan konteks global.

**FastText** dibina berdasarkan Word2Vec dengan mempelajari representasi vektor untuk setiap perkataan dan n-gram aksara yang terdapat dalam setiap perkataan. Nilai representasi ini kemudian dirata-rata menjadi satu vektor pada setiap langkah latihan. Walaupun ini menambah banyak pengiraan tambahan semasa pra-latihan, ia membolehkan embedding perkataan menyandikan maklumat sub-perkataan.

Kaedah lain, **GloVe**, menggunakan idea matriks ko-berlaku, dan menggunakan kaedah neural untuk menguraikan matriks ko-berlaku menjadi vektor perkataan yang lebih ekspresif dan tidak linear.

Anda boleh mencuba contoh ini dengan menukar embedding kepada FastText dan GloVe, kerana gensim menyokong beberapa model embedding perkataan yang berbeza.


## Menggunakan Pra-Latihan Embedding dalam PyTorch

Kita boleh mengubah contoh di atas untuk mengisi matriks dalam lapisan embedding kita dengan embedding semantik, seperti Word2Vec. Kita perlu mengambil kira bahawa perbendaharaan kata embedding pra-latihan dan korpus teks kita mungkin tidak sepadan, jadi kita akan memulakan berat untuk perkataan yang hilang dengan nilai rawak:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Sekarang mari kita latih model kita. Perhatikan bahawa masa yang diperlukan untuk melatih model adalah jauh lebih lama berbanding contoh sebelumnya, disebabkan saiz lapisan embedding yang lebih besar, dan oleh itu bilangan parameter yang jauh lebih tinggi. Selain itu, kerana perkara ini, kita mungkin perlu melatih model kita dengan lebih banyak contoh jika kita ingin mengelakkan overfitting.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Dalam kes kita, kita tidak melihat peningkatan ketepatan yang besar, yang mungkin disebabkan oleh perbezaan kosa kata yang ketara.  
Untuk mengatasi masalah perbezaan kosa kata, kita boleh menggunakan salah satu penyelesaian berikut:  
* Melatih semula model word2vec pada kosa kata kita  
* Memuatkan dataset kita dengan kosa kata daripada model word2vec yang telah dilatih. Kosa kata yang digunakan untuk memuatkan dataset boleh ditentukan semasa proses pemuatan.  

Pendekatan kedua kelihatan lebih mudah, terutamanya kerana rangka kerja PyTorch `torchtext` mengandungi sokongan terbina dalam untuk embedding. Sebagai contoh, kita boleh mencipta kosa kata berasaskan GloVe dengan cara berikut:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Perbendaharaan kata yang dimuatkan mempunyai operasi asas berikut:
* Kamus `vocab.stoi` membolehkan kita menukar perkataan kepada indeks kamusnya
* `vocab.itos` melakukan sebaliknya - menukar nombor kepada perkataan
* `vocab.vectors` adalah array vektor embedding, jadi untuk mendapatkan embedding bagi perkataan `s` kita perlu menggunakan `vocab.vectors[vocab.stoi[s]]`

Berikut adalah contoh manipulasi embedding untuk menunjukkan persamaan **kind-man+woman = queen** (saya terpaksa menyesuaikan koefisien sedikit untuk menjadikannya berfungsi):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Untuk melatih pengklasifikasi menggunakan pengekodan tersebut, kita perlu mengekod dataset kita menggunakan perbendaharaan kata GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Seperti yang telah kita lihat di atas, semua vektor embedding disimpan dalam matriks `vocab.vectors`. Ia menjadikannya sangat mudah untuk memuatkan berat tersebut ke dalam berat lapisan embedding menggunakan penyalinan mudah:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Sekarang mari kita latih model kita dan lihat jika kita mendapat hasil yang lebih baik:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Salah satu sebab kita tidak melihat peningkatan ketepatan yang ketara adalah kerana beberapa perkataan daripada set data kita tiada dalam kosa kata GloVe yang telah dilatih, dan oleh itu ia pada dasarnya diabaikan. Untuk mengatasi perkara ini, kita boleh melatih penjelmaan kita sendiri pada set data kita.


## Pembenaman Kontekstual

Satu kekurangan utama dalam representasi pembenaman pralatih tradisional seperti Word2Vec adalah masalah penyahkekeliruan makna perkataan. Walaupun pembenaman pralatih dapat menangkap sebahagian makna perkataan dalam konteks, setiap kemungkinan makna sesuatu perkataan dikodkan ke dalam pembenaman yang sama. Ini boleh menyebabkan masalah dalam model hiliran, kerana banyak perkataan seperti 'play' mempunyai makna yang berbeza bergantung pada konteks penggunaannya.

Sebagai contoh, perkataan 'play' dalam dua ayat berikut mempunyai makna yang agak berbeza:
- Saya pergi ke sebuah **play** di teater.
- John mahu **play** dengan kawan-kawannya.

Pembenaman pralatih di atas mewakili kedua-dua makna perkataan 'play' dalam pembenaman yang sama. Untuk mengatasi kekurangan ini, kita perlu membina pembenaman berdasarkan **model bahasa**, yang dilatih pada korpus teks yang besar, dan *tahu* bagaimana perkataan boleh disusun dalam pelbagai konteks. Perbincangan mengenai pembenaman kontekstual adalah di luar skop tutorial ini, tetapi kita akan kembali kepada topik ini apabila membincangkan model bahasa dalam unit seterusnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
